In [19]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np
from numpy.linalg import norm,inv,matrix_rank

import h5py
from sklearn.cross_decomposition import PLSRegression

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [20]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
 

877


In [21]:
print(X_train.shape)
print(Y_train.shape)
n_fold = 17
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break
    
print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break    

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [ ]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

def Comp_Model_Score(estimator, cv, X, Y):
    score = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(cv):
        try:
            estimator.fit(X[train_index], Y[train_index])
            y_true = Y[test_index]
            y_pred = estimator.predict(X[test_index])
            score[i] = RMSE(y_true, y_pred)
        except:
            score[i] = np.nan
            break
    return np.mean(score)

In [24]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(4,9,4))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(),  parameters,
                   cv=tscv, verbose=3,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=SIMPLS(), param_grid={'n_components': [4, 8]},
             scoring='neg_root_mean_squared_error', verbose=3)
Fitting 17 folds for each of 2 candidates, totalling 34 fits


[CV 1/17] END ..................n_components=4;, score=-4.562 total time=   0.7s


[CV 2/17] END ..................n_components=4;, score=-5.900 total time=   0.6s


[CV 3/17] END ..................n_components=4;, score=-6.039 total time=   0.5s


[CV 4/17] END ..................n_components=4;, score=-7.400 total time=   0.5s


...

[CV 14/17] END .................n_components=8;, score=-5.618 total time=   0.8s


[CV 15/17] END .................n_components=8;, score=-5.936 total time=   0.9s


[CV 16/17] END .................n_components=8;, score=-7.217 total time=   0.9s


[CV 17/17] END .................n_components=8;, score=-6.849 total time=   0.9s


CV elapsed time: 27.49s
best parameter:  {'n_components': 8} ; score:  -7.2500481e+00


In [39]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_list=list(range(4,9,4))
print("n_comp:", n_comp_list)
score_all = np.zeros((n_fold, len(n_comp_list)))

n_comp_max=10
PLS = ISIMPLS(n_components=n_comp_max)

tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    y_true = Y_train[test_index]
    for j,n_comp in enumerate(n_comp_list):
        y_pred = PLS.predict(X_train[test_index], n_comp)
        score_all[i,j] = RMSE(y_true, y_pred)
    if (i%5==0): 
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

scores = score_all.mean(axis=0)
best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {n_comp_list[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

n_comp: [4, 8]


fold: 01, elapsed time: 1.4s


fold: 06, elapsed time: 5.1s


fold: 11, elapsed time: 5.1s


fold: 16, elapsed time: 5.2s



best parameter: 8; score: 7.2500481e+00; total time=17.8s
